<a href="https://colab.research.google.com/github/leoguti/route-compare/blob/main/notebooks/notebooks00_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [12]:
PROJECT_DIR = "/content/drive/Shareddrives/Tier 2: Trufistas/Projects/GIZ CIMO PERU/Datos Rutas"
!ls "$PROJECT_DIR"


'Datos Trujillo 2024.gsheet'
 kml-por-ruta
'Listado Maestro Trujillo v2.gsheet'
'PLAN REGULADOR RUTAS - MICROS, COMBIS - OM-002-2011-MPT - Fichas Vigentes.xlsx'
 processed


In [13]:
# Ruta base del proyecto en Drive
PROJECT_DIR = "/content/drive/Shareddrives/Tier 2: Trufistas/Projects/GIZ CIMO PERU/Datos Rutas"

# Rutas específicas de interés
KML_DIR = f"{PROJECT_DIR}/kml-por-ruta"
LISTADO_2024 = f"{PROJECT_DIR}/Datos Trujillo 2024.gsheet"
LISTADO_MAESTRO_V2 = f"{PROJECT_DIR}/Listado Maestro Trujillo v2.gsheet"
PLAN_REGULADOR = f"{PROJECT_DIR}/PLAN REGULADOR RUTAS - MICROS, COMBIS - OM-002-2011-MPT - Fichas Vigentes.xlsx"
PROCESSED_DIR = f"{PROJECT_DIR}/processed"

print("PROJECT_DIR:", PROJECT_DIR)
print("KML_DIR:", KML_DIR)
print("LISTADO_2024:", LISTADO_2024)
print("LISTADO_MAESTRO_V2:", LISTADO_MAESTRO_V2)
print("PLAN_REGULADOR:", PLAN_REGULADOR)
print("PROCESSED_DIR:", PROCESSED_DIR)


PROJECT_DIR: /content/drive/Shareddrives/Tier 2: Trufistas/Projects/GIZ CIMO PERU/Datos Rutas
KML_DIR: /content/drive/Shareddrives/Tier 2: Trufistas/Projects/GIZ CIMO PERU/Datos Rutas/kml-por-ruta
LISTADO_2024: /content/drive/Shareddrives/Tier 2: Trufistas/Projects/GIZ CIMO PERU/Datos Rutas/Datos Trujillo 2024.gsheet
LISTADO_MAESTRO_V2: /content/drive/Shareddrives/Tier 2: Trufistas/Projects/GIZ CIMO PERU/Datos Rutas/Listado Maestro Trujillo v2.gsheet
PLAN_REGULADOR: /content/drive/Shareddrives/Tier 2: Trufistas/Projects/GIZ CIMO PERU/Datos Rutas/PLAN REGULADOR RUTAS - MICROS, COMBIS - OM-002-2011-MPT - Fichas Vigentes.xlsx
PROCESSED_DIR: /content/drive/Shareddrives/Tier 2: Trufistas/Projects/GIZ CIMO PERU/Datos Rutas/processed


In [14]:
import os

print("Existe carpeta kml-por-ruta?:", os.path.exists(KML_DIR))
if os.path.exists(KML_DIR):
    files = os.listdir(KML_DIR)
    print("Número de archivos KML:", len(files))
    print("Primeros 10 archivos:")
    for f in files[:10]:
        print(" -", f)


Existe carpeta kml-por-ruta?: True
Número de archivos KML: 124
Primeros 10 archivos:
 - Ruta_M-34__A_.1.kml
 - Ruta_C-22__M__vuelta.kml
 - RutaM-35_D2_.Brazo2.f.kml
 - Ruta_C-42_B2_brazo2v.kml
 - Ruta_M-36__LV_.kml
 - Ruta_C-06__K.kml
 - Ruta_M-02__A.kml
 - Ruta_M-21_A_Yahuar_Huaca.kml
 - Ruta_C-21__R__vuelta.kml
 - RutaM-23_B_.Brazo1.Alterna.kml


In [ ]:
import pandas as pd

print("Leyendo Plan Regulador...")
plan_df = pd.read_excel(PLAN_REGULADOR)

print("Filas:", len(plan_df))
plan_df.head()


In [23]:
import re
import pandas as pd
import gspread
from google.colab import auth
import google.auth # Import google.auth

# Function to extract sheet ID and GID
def get_sheet_ids(sheet_url):
    match = re.search(r"/d/([a-zA-Z0-9_-]+)", sheet_url)
    if not match:
        raise ValueError("No se pudo encontrar el ID de la hoja de cálculo en la URL.")
    sheet_id = match.group(1)

    gid_match = re.search(r"gid=(\d+)", sheet_url)
    gid = gid_match.group(1) if gid_match else "0"
    return sheet_id, gid

LISTADO_TRUJILLO_V2_URL = "https://docs.google.com/spreadsheets/d/1DqplLS5iLnz3oHqtJCCdOvpUyhWGnL5VzeyaGhb2VSA/edit?gid=0#gid=0"
DATOS_TRUJILLO_2024_URL = "https://docs.google.com/spreadsheets/d/1XKLTLMK2hgWXyGf3Qqn9n-KnHrSgSgWokk9GB8BT6lg/edit?gid=0#gid=0"

# Authenticate Google Colab to access Google Sheets
auth.authenticate_user()
# Get credentials from the authenticated user
credentials, project_id = google.auth.default()
# Initialize gspread client with the obtained credentials
gc = gspread.Client(auth=credentials)

# Read LISTADO_TRUJILLO_V2
print("Reading LISTADO_TRUJILLO_V2...")
listado_v2_sheet_id, listado_v2_gid = get_sheet_ids(LISTADO_TRUJILLO_V2_URL)
listado_v2_spreadsheet = gc.open_by_key(listado_v2_sheet_id)
listado_v2_worksheet = listado_v2_spreadsheet.get_worksheet_by_id(int(listado_v2_gid)) # gspread needs GID as int
listado_v2_data = listado_v2_worksheet.get_all_records()
listado_v2_df = pd.DataFrame.from_records(listado_v2_data)

# Read DATOS_TRUJILLO_2024
print("Reading DATOS_TRUJILLO_2024...")
datos_2024_sheet_id, datos_2024_gid = get_sheet_ids(DATOS_TRUJILLO_2024_URL)
datos_2024_spreadsheet = gc.open_by_key(datos_2024_sheet_id)
datos_2024_worksheet = datos_2024_spreadsheet.get_worksheet_by_id(int(datos_2024_gid))

# Get all values from the worksheet to manually handle headers
all_values = datos_2024_worksheet.get_all_values()

if all_values:
    # Assume the first row is the header
    headers = all_values[0]
    # Process headers to ensure uniqueness, especially for empty strings
    processed_headers = []
    seen_headers = {}
    for h in headers:
        original_h = h.strip() if h else '' # Clean up and handle truly empty strings
        if original_h in seen_headers:
            seen_headers[original_h] += 1
            processed_headers.append(f"{original_h}_{seen_headers[original_h]}")
        else:
            seen_headers[original_h] = 0 # Initialize counter for first occurrence
            processed_headers.append(original_h)

    # The rest of the rows are data
    datos_2024_data = all_values[1:]
    datos_2024_df = pd.DataFrame(datos_2024_data, columns=processed_headers)
else:
    datos_2024_df = pd.DataFrame() # Create an empty DataFrame if no data

print("Successfully read both Google Sheets.")
display(listado_v2_df.head())
display(datos_2024_df.head())

Reading LISTADO_TRUJILLO_V2...
Reading DATOS_TRUJILLO_2024...
Successfully read both Google Sheets.


,Denominación,Nº,Código de Ruta,Letra de Ruta,Tipo,KML,Estado,Observaciones
0,CALIFORNIA S.A.,1,M-01,C,Omnibus,Ruta_M-01__C.kml,Incluida,
1,CALIFORNIA S.A.,2,M-02,A,Omnibus,Ruta_M-02__A.kml,Incluida,
2,CALIFORNIA S.A.,3,M-03,B,Omnibus,Ruta_M-03_B_ida.kml,Incluida,
3,CALIFORNIA S.A.,3,M-03,B,Omnibus,Ruta_M-03_B_vuelta.kml,Incluida,
4,HUANCHACO S.A.,4,M-04,A,Omnibus,Ruta_M-04__A.kml,Incluida,


,Codigo,Nombre,Referencia (Ref),Desde (From),A (To),Circuito (roundtrip),GeoJSON/ GPX / KML/ KMZ Archivo,Horario de servicio (opening_hours),Frecuencia en minutos (Interval),Info Horas Pico (Interval conditional),...,Auto Validation,OSM Relation,,_1,_2,_3,_4,_5,_6,_7
0,Como se conoce en el lugar,"Nombre completo de la ruta, es una formula no ...","La ruta se conoce por esta referencia, por eje...",Donde inicia el recorrido,Donde acaba el recorrido,Para indicar que el inicio y el final de la ru...,"Enlace al archivo de la ruta, en cualquiera de...",Ingrese la hora en que comienza a brindarse el...,Se obtiene preguntando a los conductores o enc...,"Registrar la frecuencia en las horas pico, los...",...,In this box is checked if the relationship is ...,Relación en OSM,,,,,,,,
1,"C-01 Letra ""B""","C-01 ""B"" : Av. Miguel Grau → Av. Miguel Grau","C-01 ""B""",Av. Miguel Grau,Av. Miguel Grau,TRUE,https://drive.google.com/file/d/1cx1OvA9vn0rtz...,Mo-Su 05:00-23:00,4,00:04 Mo-Fr 06:00-07:00 12:00-13:30: 16:30-18:00,...,COMPLETED,17350695,http://localhost:8111/load_object?objects=rela...,,,,,,,
2,"C-02 Letra ""H""","C-02 ""H"" : Ca. Los Olivares Mz. 7 - Lte. 1 → C...","C-02 ""H""",Ca. Los Olivares Mz. 7 - Lte. 1,Ca. Los Olivares Mz. 7 - Lte. 1,TRUE,https://drive.google.com/file/d/1Zwb8IqscG2IAt...,,,,...,,,,,,,,,,
3,"C-03 Letra ""JJ""","C-03 ""JJ"" : Av. César Vallejo → Av. César Vallejo","C-03 ""JJ""",Av. César Vallejo,Av. César Vallejo,TRUE,https://drive.google.com/file/d/1kTyDK9NcIsGQQ...,Mo-Su 04:30-18:00,5,00:04 Mo-Fr 06:00-07:00: 12:00-13:30: 16:30-18:00,...,#REF!,17491518,http://localhost:8111/load_object?objects=rela...,,,,,,,
4,"C-04 Letra ""XS""","C-04 ""XS"" : Ca. Los Olivares S/N Mz. 7 - Lte. ...","C-04 ""XS""",Ca. Los Olivares S/N Mz. 7 - Lte. 1,Ca. Los Olivares S/N Mz. 7 - Lte. 1,TRUE,https://drive.google.com/file/d/1HWvRbfVSetOPD...,,,,...,,,,,,,,,,


In [25]:
import os

# Ensure the processed directory exists
os.makedirs(PROCESSED_DIR, exist_ok=True)

# Define the output CSV file path
output_csv_path = os.path.join(PROCESSED_DIR, "Datos_Trujillo_2024.csv")

# Save the DataFrame to a CSV file
datos_2024_df.to_csv(output_csv_path, index=False, encoding='utf-8')

print(f"'Datos Trujillo 2024' ha sido convertido y guardado en: {output_csv_path}")

'Datos Trujillo 2024' ha sido convertido y guardado en: /content/drive/Shareddrives/Tier 2: Trufistas/Projects/GIZ CIMO PERU/Datos Rutas/processed/Datos_Trujillo_2024.csv


In [26]:
print(list(listado_v2_df.columns))
print(list(datos_2024_df.columns))


['Denominación', 'Nº', 'Código de Ruta', 'Letra de Ruta', 'Tipo ', 'KML', 'Estado', 'Observaciones']
['Codigo', 'Nombre', 'Referencia (Ref)', 'Desde (From)', 'A (To)', 'Circuito (roundtrip)', 'GeoJSON/ GPX / KML/ KMZ Archivo', 'Horario de servicio (opening_hours)', 'Frecuencia en minutos (Interval)', 'Info Horas Pico (Interval conditional)', 'Operador (Operator)', 'Color', 'Network', 'Distancia de recorrido', 'Tiempo de recorrido (Estimated full travel time)', 'Precio (Charge)', 'Foto de la unidad', 'Foto del tarifario', 'Notas', 'ID de la Traza', 'Enlace traza OSM', 'Responsable Recoleccion', 'Horas recoleccion', 'Horas Decimales', 'Lista para trabajar', 'Id', 'Owner', 'Finished', 'Local Check', 'Paradas', 'Trufi tool', 'Validator', 'JOSM \nlink', 'Auto Validation', 'OSM Relation', '', '_1', '_2', '_3', '_4', '_5', '_6', '_7']
